# Initial State Configuration for FCFS Queue with Non-Exponential Service

This example demonstrates initial state configuration for a multi-class closed network with FCFS scheduling, class switching, and non-exponential service distributions.

In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.mva.*
import jline.solvers.nc.*
import jline.solvers.jmt.*
import jline.util.matrix.*

In [ ]:
val model = Network("model")

// Create nodes
val delay = Delay(model, "Delay")
val queue1 = Queue(model, "Queue1", SchedStrategy.FCFS)
queue1.setNumberOfServers(3)  // Multi-server queue

// Create two closed classes with different populations
val class1 = ClosedClass(model, "Class1", 3, queue1, 0)  // 3 jobs of Class1 start at Queue1
val class2 = ClosedClass(model, "Class2", 2, queue1, 0)  // 2 jobs of Class2 start at Queue1

println("Model Setup:")
println("- Class 1: ${class1.population} jobs")
println("- Class 2: ${class2.population} jobs")
println("- Total jobs: ${class1.population + class2.population}")
println("- Queue1 servers: ${queue1.numberOfServers}")
println("- Initial state: All jobs start at Queue1")

In [ ]:
// Set service processes with mixed distributions
delay.setService(class1, Exp(1.0))  // Class1: Exponential service at Delay
delay.setService(class2, Exp(1.0))  // Class2: Exponential service at Delay

queue1.setService(class1, Exp(1.2))                         // Class1: Exponential service at Queue1
queue1.setService(class2, Erlang.fitMeanAndSCV(1.0, 0.5))  // Class2: Erlang service at Queue1

println("\nService Configurations:")
println("Delay Station:")
println("  - Class1: Exponential(μ=1.0)")
println("  - Class2: Exponential(μ=1.0)")
println("Queue1 Station:")
println("  - Class1: Exponential(μ=1.2)")
println("  - Class2: Erlang(mean=1.0, SCV=0.5)")

In [ ]:
// Set up complex routing matrix with class switching
val P = model.initRoutingMatrix()

// Class 1 to Class 1 routing
P.set(class1, class1, delay, delay, 0.3)    // Stay at Delay with prob 0.3
P.set(class1, class1, delay, queue1, 0.7)   // Move to Queue1 with prob 0.7
P.set(class1, class1, queue1, delay, 0.2)   // Move to Delay with prob 0.2
P.set(class1, class1, queue1, queue1, 0.0)  // Cannot stay at Queue1

// Class 1 to Class 2 routing (class switching)
P.set(class1, class2, delay, delay, 0.0)
P.set(class1, class2, delay, queue1, 0.0)
P.set(class1, class2, queue1, delay, 0.8)   // Switch to Class2 and move to Delay
P.set(class1, class2, queue1, queue1, 0.0)

// Class 2 to Class 2 routing
P.set(class2, class2, delay, delay, 0.0)
P.set(class2, class2, delay, queue1, 1.0)   // Always move to Queue1
P.set(class2, class2, queue1, delay, 0.0)
P.set(class2, class2, queue1, queue1, 0.0)

// Class 2 to Class 1 routing (class switching back)
P.set(class2, class1, delay, delay, 0.0)
P.set(class2, class1, delay, queue1, 0.0)
P.set(class2, class1, queue1, delay, 1.0)   // Switch to Class1 and move to Delay
P.set(class2, class1, queue1, queue1, 0.0)

model.link(P)

println("\nRouting Configuration:")
println("Class1 dynamics:")
println("  - 30% probability to loop at Delay")
println("  - 70% probability to move Delay→Queue1")
println("  - 20% probability to stay as Class1 and move Queue1→Delay")
println("  - 80% probability to switch to Class2 and move Queue1→Delay")
println("Class2 dynamics:")
println("  - Always moves Delay→Queue1")
println("  - Always switches to Class1 and moves Queue1→Delay")

In [ ]:
// Solve with MVA
try {
    val solverMva = MVA(model)
    val avgTableMva = solverMva.avgTable
    println("\n=== MVA Results ===")
    avgTableMva.print()
} catch (e: Exception) {
    println("\nMVA solver failed: ${e.message}")
    println("This may be due to the complex class switching or non-exponential distributions")
}

In [ ]:
// Solve with NC (Normalizing Constant)
try {
    val solverNc = NC(model)
    val avgTableNc = solverNc.avgTable
    println("\n=== NC Results ===")
    avgTableNc.print()
} catch (e: Exception) {
    println("\nNC solver failed: ${e.message}")
}

In [ ]:
// Solve with JMT simulation
try {
    val jmtOptions = JMT.defaultOptions()
    jmtOptions.samples = 10000
    
    val solverJmt = JMT(model, jmtOptions)
    val avgTableJmt = solverJmt.avgTable
    println("\n=== JMT Simulation Results ===")
    avgTableJmt.print()
} catch (e: Exception) {
    println("\nJMT solver failed: ${e.message}")
}

In [ ]:
// Analysis of the complex system behavior
println("\n=== System Analysis ===")
println("This model demonstrates several advanced features:")
println("\n1. Initial State Configuration:")
println("   - All 5 jobs (3 Class1 + 2 Class2) start at Queue1")
println("   - Creates initial congestion at the multi-server queue")
println("\n2. Class Switching Dynamics:")
println("   - Class1 jobs can switch to Class2 (80% probability at Queue1)")
println("   - Class2 jobs always switch back to Class1 at Queue1")
println("   - Creates a complex circulation pattern")
println("\n3. Non-Exponential Service:")
println("   - Class2 uses Erlang distribution (lower variability)")
println("   - Class1 uses Exponential distribution")
println("   - Different service variabilities affect queueing behavior")
println("\n4. Multi-Server Queue:")
println("   - Queue1 has 3 servers to handle initial congestion")
println("   - Helps process the initial job accumulation")
println("\n5. Transient Behavior:")
println("   - System starts highly unbalanced (all jobs at Queue1)")
println("   - Jobs gradually redistribute through class switching")
println("   - Reaches steady state with balanced class populations")
println("\nThis type of model is relevant for:")
println("- Manufacturing systems with job type changes")
println("- Computer systems with process state transitions")
println("- Service systems with customer type switching")
println("- Recovery analysis after system restarts")

This example demonstrates:

1. **Complex Initial State**: 
   - Multi-class system with all jobs starting at one location
   - Creates initial imbalance that must be resolved through routing
2. **Class Switching Network**:
   - Jobs can change class during their journey through the network
   - Class1 ↔ Class2 transitions create complex dynamics
   - Different routing probabilities for each class
3. **Mixed Service Distributions**:
   - Exponential service for most processes
   - Erlang service for Class2 at Queue1 (reduced variability)
   - Tests solver capabilities with non-exponential distributions
4. **Multi-Server Configuration**:
   - Queue1 has 3 servers to handle potential congestion
   - Important for managing initial job accumulation
5. **Solver Comparison**:
   - Tests multiple solution methods (MVA, NC, JMT)
   - Some solvers may have limitations with complex routing

This model is particularly useful for analyzing:
- System recovery from non-equilibrium states
- Impact of job type switching on performance
- Transient behavior in complex manufacturing or service systems
- Validation of different solution approaches